In [1]:
import pandas as pd
import matplotlib.pyplot as plt

Lectura de archivos

In [2]:
tweets_df = pd.read_csv("../data/tweets_with_sentiment_cryptobert.csv")
tweets_df['Created At'] = pd.to_datetime(tweets_df['Created At'], errors='coerce')
tweets_df['fecha'] = tweets_df['Created At'].dt.date

price_df = pd.read_csv("../data/datos_mercado_binance.csv")
price_df['fecha'] = pd.to_datetime(price_df['fecha'], errors='coerce')

In [3]:
# We count by kind of feeling
conteo_sentimientos = tweets_df.groupby(['fecha', 'Sentiment']).size().unstack(fill_value=0)

# We recalculate totals and average
conteo_sentimientos['numero_noticias'] = conteo_sentimientos.sum(axis=1)
conteo_sentimientos['promedio_sentimiento'] = tweets_df.groupby('fecha')['Sentiment'].mean()

Transforma las columnas date en formato datetime.date

In [4]:
price_df['fecha'] = pd.to_datetime(price_df['fecha']).dt.date 
conteo_sentimientos = conteo_sentimientos.reset_index()  # si 'fecha' está en el índice
conteo_sentimientos['fecha'] = pd.to_datetime(conteo_sentimientos['fecha']).dt.date

Realiza la fusión de los dos DataFrames (price_df y conteo_sentimientos) usando la columna fecha como clave



In [5]:
# We join by date
df_final = price_df.merge(conteo_sentimientos, on='fecha', how='left')

CALCULO MEDIA MOVIL CON DERIVADA Y TENDENCIA

In [6]:

# We added trend column
window = 10
df_final['media_movil_10d'] = df_final['precio_apertura'].rolling(window).mean()
df_final['derivada_mm10'] = df_final['media_movil_10d'].diff()
def clasificar_derivada(valor):
    if pd.isna(valor):
        return 0  
    elif valor > 0:
        return 1
    elif valor < 0:
        return -1
    else:
        return 0
df_final['tendencia_categorizada'] = df_final['derivada_mm10'].apply(clasificar_derivada)


📈 Indicador RSI (Relative Strength Index)

📊 RSI toma valores entre 0 y 100:

- (<30) podría indicar sobreventa.
- (>70) podría indicar sobrecompra.

In [13]:
def calcular_rsi(series, window=10):
    delta = series.diff()
    ganancias = delta.clip(lower=0)
    perdidas = -delta.clip(upper=0)

    media_ganancia = ganancias.rolling(window).mean()
    media_perdida = perdidas.rolling(window).mean()

    rs = media_ganancia / media_perdida
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_final['RSI_10'] = calcular_rsi(df_final['precio_apertura'])

🔁 Indicador MACD (Moving Average Convergence Divergence)


In [14]:
# EMA rápida y lenta
ema_12 = df_final['precio_apertura'].ewm(span=12, adjust=False).mean()
ema_26 = df_final['precio_apertura'].ewm(span=26, adjust=False).mean()

# Línea MACD y señal
df_final['MACD'] = ema_12 - ema_26
df_final['MACD_signal'] = df_final['MACD'].ewm(span=9, adjust=False).mean()

In [ ]:
# Selecting final columns
df_csv = df_final[['fecha', 'precio_apertura','media_movil_10d', 'RSI_10', 'MACD_signal', 'tendencia_categorizada', 'numero_noticias', -1, 0, 1, 'promedio_sentimiento']]

# We rename columns
df_csv.columns = ['DATE', 'OPEN','MA','RSI','MACD', 'TREND', 'NEWS ISSUE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'AVERAGE NEWS']

# We save CSV
df_csv.to_csv('../data/analisis_completo.csv', index=False)